In [ ]:
using JuAFEM
using ContMechTensors

In [ ]:
import Base.flatten

In [ ]:
const dim = 3
const grid = generate_grid(Tetrahedron, (40,15,15), Vec{dim}((0.0, -1.0, -1.0)), Vec{dim}((10.0, 1.0, 1.0)));
nnodes = getnnodes(grid)
ncells = getncells(grid)

In [ ]:
const interpolation_space = Lagrange{dim, RefTetrahedron, 1}()
const quadrature_rule = QuadratureRule{dim, RefTetrahedron}(1)
const cellvalues = CellVectorValues(quadrature_rule, interpolation_space);
const facevalues = BoundaryVectorValues(QuadratureRule{dim-1, RefTetrahedron}(1), interpolation_space);

In [ ]:
# Create Node dof matrix
const nodedofs = reshape(1:nnodes * dim, (dim, nnodes));

In [ ]:
# Create "edof" matrix
const edofs = zeros(Int, getnbasefunctions(cellvalues), ncells)
for (i, cell) in enumerate(getcells(grid))
    edofs[:, i] = collect(flatten([nodedofs[:, v] for v in getvertices(grid, cell)]))
end

In [ ]:
# Extract the left boundary
left_boundarynodes = []
left = getcellboundaryset(grid, "left")
for boundary in getboundaries(grid, left)
    append!(left_boundarynodes, getnodes(grid, boundary))
end
left_boundarynodes = unique(left_boundarynodes);

In [ ]:
dofs_left_boundary  = collect(flatten(nodedofs[:, i] for i in left_boundarynodes));

In [ ]:
E = 200e9
ν = 0.3
λ = E*ν / ((1+ν) * (1 - 2ν))
μ = E / (2(1+ν))
δ(i,j) = i == j ? 1.0 : 0.0
g(i,j,k,l) = λ*δ(i,j)*δ(k,l) + μ*(δ(i,k)*δ(j,l) + δ(i,l)*δ(j,k))
C = SymmetricTensor{4, dim}(g);

In [ ]:
function doassemble{dim}(cellvalues::CellVectorValues{dim}, facevalues::BoundaryVectorValues{dim},
    grid::Grid, C::SymmetricTensor{4, dim}, edofs, nodedofs)
    n_basefuncs = getnbasefunctions(cellvalues)
    Ke = zeros(n_basefuncs, n_basefuncs)
    
    ɛ = [zero(SymmetricTensor{2, dim}) for i in 1:n_basefuncs]
    assembler = start_assemble()
    @inbounds for (cellcount, cell) in enumerate(getcells(grid))
        fill!(Ke, 0)
        reinit!(cellvalues, getcoordinates(grid, cell))
        for q_point in 1:getnquadpoints(cellvalues)
            for i in 1:n_basefuncs
                ɛ[i] = symmetric(shape_gradient(cellvalues, q_point, i)) 
            end
            dΩ = getdetJdV(cellvalues, q_point)
            for i in 1:n_basefuncs
                ɛC = ɛ[i] ⊡ C
                for j in 1:n_basefuncs
                    Ke[i, j] += (ɛC ⊡ ɛ[j]) * dΩ
                end
            end
        end
        assemble!(assembler, Ke, edofs[:, cellcount])
    end
    K = end_assemble(assembler)
   
    n_facefuncs = n_basefuncs - dim # Awkward
    f = zeros(length(nodedofs))
    fe = zeros(n_basefuncs - dim)
    t = Vec{3}((0.0, 1e8, 0.0))
    global_dofs = zeros(Int, n_facefuncs)
    for boundaryidx in getcellboundaryset(grid, "right")
        boundary = getboundaries(grid, boundaryidx)
        fill!(fe, 0)
        coords = getcoordinates(grid, JuAFEM.CellIndex(boundary.idx[1])) # Awkward
        reinit!(facevalues, coords, boundary.idx[2]) # Awkward (make reinit!(facevalues, grid, boundaryindex) work)
         for q_point in 1:getnquadpoints(cellvalues)
            for i in 1:n_facefuncs
                N = shape_value(facevalues, q_point, i)
                dΩ = getdetJdV(facevalues, q_point)
                fe[i] += (N ⋅ t) * dΩ
            end
        end
        
        # The whole thing below is a bit awkward,
        # better as getdofs!(global_dofs, dofhandler, grid, boundary) or something.
        resize!(global_dofs, 0)
        nodes_in_face = getnodes(grid, boundary)
        for n in nodes_in_face
            append!(global_dofs, nodedofs[:, n])
        end
        f[global_dofs] += fe
    end
    return K, f
end

In [ ]:
K, f = doassemble(cellvalues, facevalues, grid, C, edofs, nodedofs);

In [ ]:
d_pres = dofs_left_boundary
# Zero dbc on left edge, one on right edge
a_pres = zeros(dofs_left_boundary)
d_free = setdiff(1:maximum(nodedofs), d_pres);

In [ ]:
# Account for boundary conditions and solve
u = zeros(maximum(nodedofs))
@time u_free = cholfact(Symmetric(K[d_free, d_free])) \ (f[d_free] - K[d_free, d_pres] * a_pres)
u[d_free] = u_free 
u[d_pres] = a_pres;

In [17]:
# Save file
vtkfile = vtk_grid("cantilever2", grid);
vtk_point_data(vtkfile, reshape(u, (dim, nnodes)), "Displacement")
vtk_save(vtkfile)

1-element Array{String,1}:
 "cantilever2.vtu"

In [ ]:
println("Cantilever successful")